In [1]:
import os
os.chdir("../")
os.getcwd()

'/home/izam/coding/hackathon'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PromptingConfig:
    root_dir: Path
    model_name: str
    response_file_name: str
    candidates_file_name: str
    credentials: Path
    generation_config: dict
    project_name: str
    project_location: str
    prompt_file_path: Path

In [3]:
from gemInsights.constants import *
from gemInsights.utils.common import read_yaml, create_directories, load_json

In [4]:
#udpate configuration manager
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        credentials_file_path = CREDENTIALS_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.credentials = credentials_file_path
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])


    def get_promting_config(self) -> PromptingConfig:
        config = self.config.prompting
        
        create_directories([config.root_dir])

        prompting_config = PromptingConfig(
            root_dir=config.root_dir,
            model_name=config.model_name, 
            response_file_name=config.response_file_name,
            candidates_file_name=config.candidates_file_name,
            credentials=self.credentials,
            generation_config=dict(self.params.generation_config),
            project_name=config.project_name,
            project_location=config.project_location,
            prompt_file_path=config.prompt_file_path,          
        )

        return prompting_config

In [5]:
from google.cloud import aiplatform
from gemInsights import logger
import os
import base64
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
from gemInsights.utils.common import load_json, save_json, load_bin

In [6]:
class Prompting:
    def __init__(self, config: PromptingConfig):
        self.config = config

    def _setup_env(self):
        aiplatform.init(
            project = self.config.project_name,
            location= self.config.project_location,
        )
        logger.info(f"Google cloud project name - {self.config.project_name}")
        
        os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = self.config.credentials
        logger.info("loaded the google cloud credentials")


    def get_response(self):
        self._setup_env()
        model = GenerativeModel(self.config.model_name)
        logger.info(f"using the model - {self.config.model_name}")

        logger.info(f"generating response with config - {self.config.generation_config}")
        prompt = load_bin(Path(self.config.prompt_file_path))
        responses = model.generate_content(
            prompt,
            generation_config=self.config.generation_config,
            )
        
        save_json(path=os.path.join(self.config.root_dir, self.config.response_file_name), data={"response": responses.text})
        logger.info(responses.text)

In [7]:
try:
    config = ConfigurationManager()
    prompting_config = config.get_promting_config()
    prompting_config = Prompting(config=prompting_config)
    prompting_config.get_response()
except Exception as e:
    raise e

[2023-12-21 13:55:05,128: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-21 13:55:05,130: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-21 13:55:05,130: INFO: common: created directory at: artifacts]
[2023-12-21 13:55:05,130: INFO: common: created directory at: artifacts/prompting]
[2023-12-21 13:55:05,131: INFO: 3427468588: Google cloud project name - ultra-heading-407815]
[2023-12-21 13:55:05,131: INFO: 3427468588: loaded the google cloud credentials]
[2023-12-21 13:55:05,131: INFO: 3427468588: using the model - gemini-pro-vision]
[2023-12-21 13:55:05,132: INFO: 3427468588: generating response with config - {'max_output_tokens': 2048, 'temperature': 0.4, 'top_p': 1, 'top_k': 32}]
[2023-12-21 13:55:05,133: INFO: common: binary file loaded from: artifacts/prompt_generation/prompt.joblib]
[2023-12-21 13:55:16,157: INFO: common: json file saved at: artifacts/prompting/response.json]
[2023-12-21 13:55:16,158: INFO: 3427468588:  EDA Univariat

In [8]:
import joblib
a = joblib.load("./artifacts/prompt_generation/prompt.joblib")
a

['Tell what are this pictures',
 inline_data {
   mime_type: "image/jpeg"
   data: "\377\330\377\340\000\020JFIF\000\001\001\001\000d\000d\000\000\377\333\000C\000\010\006\006\007\006\005\010\007\007\007\t\t\010\n\014\024\r\014\013\013\014\031\022\023\017\024\035\032\037\036\035\032\034\034 $.\' \",#\034\034(7),01444\037\'9=82<.342\377\333\000C\001\t\t\t\014\013\014\030\r\r\0302!\034!22222222222222222222222222222222222222222222222222\377\300\000\021\010\007\320\005\334\003\001\"\000\002\021\001\003\021\001\377\304\000\037\000\000\001\005\001\001\001\001\001\001\000\000\000\000\000\000\000\000\001\002\003\004\005\006\007\010\t\n\013\377\304\000\265\020\000\002\001\003\003\002\004\003\005\005\004\004\000\000\001}\001\002\003\000\004\021\005\022!1A\006\023Qa\007\"q\0242\201\221\241\010#B\261\301\025R\321\360$3br\202\t\n\026\027\030\031\032%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\203\204\205\206\207\210\211\212\222\223\224\225\226\227\230\231\232\242\243\244\245\246\247\250\251\252\2